## Autonomy equations for data-set from people in MSA  

There are 3 types of data in a data-set in Autonomy. They are confirmed cases from Taiwan CDC, symptom reports, and good behavior reports from Autonomy users in a MSA area. Follows are how we calculate these indicators


### Symptom Indicator
+ There are two types of symptoms: **Official** & **Customized** Symptom.
+ A user can adjust Weight for each symptom to have their score
+ The higher number symptoms reported, the lower the score
+ The score of symptoms is:

```Score = 100 * (1 -(SumOfTotalWeight/((TotalPeople*MaxWeightPerPerson)+SumOfCustomizedWeight)))```

+ Factors
	+ SumOfTotalWeight
		+ WeightMatrix : the weight that a user gives to each official symptom.
		```
		{
			Fever:   3,
			Cough:   2,
			Fatigue: 1,
			Breath:  1,
			Nasal:   1,
			Throat:  1,
			Chest:   2,
			Face:    2,
		}
		```
	+ TotalPeople : total people report in the MSA at specific period (ie. today) 
	+ MaxWeightPerPerson: the weights a person has, if the person reports all official symptoms
		+ use the WeightMatrix above, it is (3+2+1+1+1+1+2+2) = 13
	+ CustomizedWeight : the sum of total customized weights
	+ SumOfCustomizedWeight: CustomizedWeight for each symptom is 1 (so CustomizedWeight = CustomizedCount)
	+ Normalization: the score is 100 and the more symptoms and the lower the score
    
	 ```
		100 - 100*(WeightsReported/WeightWeExpectedAsA100Point)
		100*(1-(WeightsReported/WeightWeExpectedAsA100Point))
		100*(1-(SumOfTotalWeight/(((TotalPeople*MaxWeightPerPerson)+CustomizedWeight)))
	```

#### Example data-set of symptom report

In  a MSA area there are 5 reports from A,B,C and D

```
 A : Fever , customized01
 B : Fever, Dry Cought, Chest Pain
 C : Dry Cough, Fatique
 D : customized02
```


Data set will have below value
+ total count of customized symptoms
+ total people in the report
+ total count of each official symptom in the Data-set (we call it distribution)
+ weight metrix : user can define the weight for each official symptoms while customized weight always weight 1

In [62]:
totalCustomizedCountSymptom = 2
totalPeopleSymptom = 4

In [63]:

distributionSymptom = {
    'Fever':   2,
    'Cough':   2,
    'Fatigue': 1,
    'Breath':  0,
    'Nasal':   0,
    'Throat':  0,
    'Chest':   1,
    'Face':    0,
}


In [64]:

weightMatrixSymptom = {
    'Fever':   3,
    'Cough':   2,
    'Fatigue': 1,
    'Breath':  1,
    'Nasal':   1,
    'Throat':  1,
    'Chest':   2,
    'Face':    2,
}


#### Equation
+ calculate intermediate values for the final score

In [65]:
totalWeightSymptom = 0.0
for key, value  in distributionSymptom.items():
    totalWeightSymptom = totalWeightSymptom + value*weightMatrixSymptom[key]

print('totalWeightSymptom',totalWeightSymptom)
maxWeightPerPersonSymptom = 0.0
for key, value  in weightMatrixSymptom.items():
    maxWeightPerPersonSymptom = maxWeightPerPersonSymptom + value
print('maxWeightPerPersonSymptom:',maxWeightPerPersonSymptom)

totalWeightSymptom 13.0
maxWeightPerPersonSymptom: 13.0


#### Final Score

In [66]:
if ((totalWeightSymptom*maxWeightPerPersonSymptom)+totalCustomizedCountSymptom*1) > 0:
    scoreSymptom = 100*(1-(totalWeightSymptom/((totalPeopleSymptom*maxWeightPerPersonSymptom)+totalCustomizedCountSymptom*1)))
else:
    scoreSymptom = 100
print('**scoreSymptom', scoreSymptom)

**scoreSymptom 75.92592592592592


###  Behaviors Indicator

+ There are two types of behaviors: **Official** & **Customized** behavior.
+ The user can not change their **weight matrix** in Sprint3
+ Each Official behavior weight 1 , the same as customized behavior
+ The more good behaviors reported, the better the score
+ The score of symptoms is:

```Score = 100*(SumOfTotalWeight/((TotalPeople*MaxWeightPerPerson)+SumOfCustomizedWeight))```

#### Example data-set of behavior report

    ```
     A : CleanHand , customized01
	 B : SocialDistancing, TouchFace
	 C : CleanHand
	 D : customized02
	```


behavior data in the autonomy data-set will have values:
+ sum of total weights (official+customized good behaviors) 
+ total people in the report
+ a max weight per person can have
+ total customized weight 

#### Equation & final score


In [67]:
# SumOfTotalWeight = 2 + 2 + 1 + 1
SumOfTotalWeightBehavior = 6
TotalPeopleBehavior = 4 
# (We have 6 official behaviors)
MaxWeightPerPersonBehavior = 6 
SumOfCustomizedWeightBehavior = 2
if ((TotalPeopleBehavior*MaxWeightPerPersonBehavior)+SumOfCustomizedWeightBehavior) > 0:
    scoreBehavior = 100*(SumOfTotalWeightBehavior/((TotalPeopleBehavior*MaxWeightPerPersonBehavior)+SumOfCustomizedWeightBehavior))
else:
    scoreBehavior = 0
print('**scoreBehavior:', scoreBehavior)

**scoreBehavior: 23.076923076923077


## Confirmed cases indicator
+ The confirmed cases data is from Taiwan CDC everydata and indicates in **county level**
+ Score Equation
    + if increased confirm cases (compare to previous day) < 10
    `Score = 100 - 5 * increased confirmed case count`

    + if increased confirm cases (compare to previous day) >= 10
    `Score = 0`


### Example
Today's confirms cases in Taipei city is 5

In [68]:
confirmsCasesToday=5

#### Equation & final score

In [69]:
scoreConfirm=100-5*5
print('ScoreConfirm:',scoreConfirm)

ScoreConfirm: 75


##  Overall score
Assume we can have normalized score ([0-100]) from each indicator above, we want to come up with an overall score for users as the final indicator. The most straighfoward way could be a linear combination:

`final socre = c1*symptom_score + c2*behavior_score + c3*confirm_case_score`

### Example
Coefficients for each score is
    
    coefficient1 = 0.25 (symptom)
    coefficient2 = 0.25 (behavior)
    coefficient13 = 0.50 (confirmed cases)
    
We use scoreSymptom , scoreBehavior and scoreConfirm from Above

In [70]:
coefSymptom=0.25
coefBehavior=0.25
coefConfirm=0.5
score= coefSymptom*scoreSymptom+ coefBehavior*scoreBehavior + coefConfirm*scoreConfirm
print('**Autonomy Overall Score:',score)

**Autonomy Overall Score: 62.250712250712255
